In [2]:
import os
import sys
import math
import re
import time
import functools
import itertools
from abc import ABC
from dataclasses import dataclass, field
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from scipy.linalg import solve
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt
!{sys.executable} -m pip install pyfinance -q
from pyfinance.options import BSM as BSMAux

Add any general utility functions as static member functions to this class. They can be called as `Util.fn`.

In [3]:
class Util:
    def __init__(self):
        raise TypeError("Non-instantiable class")
        
    def __new__(self, *args, **kwargs):
        raise TypeError("Non-instantiable class")
        
    @staticmethod
    def make_regex_group_disjunction(coll):
        return "|".join(map(lambda x: f"({str(x)})", coll))

In [4]:
class OptionsData:
    data_dir = "data"
    default_filename = "isx2010C.xls"
    
    def __init__(self, filename=default_filename, clean=True):
        filepath = os.path.join(self.data_dir, filename)
        if not os.path.isfile(filepath):
            faulty_filepath = filepath
            filepath = os.path.join(self.data_dir, self.default_filename)
            print(f"[{type(self).__name__}] Warning: could not find {faulty_filepath!r}; proceeding with {filepath!r}")
        self.__sheet_df_dict = pd.read_excel(filepath, sheet_name=None)
        sheets = list(self.__sheet_df_dict.keys())
        self.__sheet_succ = dict(zip(sheets, sheets[1:] + [sheets[-1]]))
        if clean:
            for key, val in self.__sheet_df_dict.items():
                self.__sheet_df_dict[key] = self.__clean_df(val)
                
    def __get_item__(self, key):
        return self.__sheet_df_dict[key]
    
    def get_sheet_names(self):
        return list(self.__sheet_succ.keys())
    
    def get_sheet_df_dict(self):
        return self.__sheet_df_dict
    
    def get_next_sheet_name(self, sheet_name):
        assert sheet_name in self.__sheet_succ
        return self.__sheet_succ[sheet_name]
    
    def get_df(self, E=None, sheet_name=""):
        if not sheet_name:
            sheet_name = list(self.__sheet_df_dict.keys())[0]
            print(f"[{type(self).__name__}] Warning: sheet name not specified; proceeding with {sheet_name!r}")
        df = self.__sheet_df_dict[sheet_name]
        common = ["T", "T_norm", "S", "r"]
        if not E:
            return df[[*common, *filter(lambda x: re.match(r"[0-9]+", x), df.columns)]]
        strikes = E if type(E) is list or type(E) is tuple else [E]
        cols = [*common, *map(lambda x: str(int(x)), strikes)]
        return df[cols]
    
    def __clean_df(self, df):
        # Discard rows where no options data is available.
        df = df.dropna(how="all")
        # Rename the columns according to the following convention:
        #  T = Time to Maturity
        #  S = Price of the Underlying
        #  r = Risk-Free Interest Rate
        df = df.rename(lambda x: self.__rename_df_cols(str(x), df), axis="columns")
        # Adjust the interest rate properly.
        df["r"] = df["r"] / 100
        # Add new column with annual-normalized T (252 = no. trading days in a year).
        df["T_norm"] = df["T"] / 252
        # Re-arrange the columns.
        common = ["S", "r", "T", "T_norm"]
        cols = [*common, *filter(lambda x: re.search("[0-9]+", x), df.columns.astype(str))]
        return df[cols]
    
    def __rename_df_cols(self, col_name, df):
        ncol = len(df.columns)
        # Time to maturity | (price of the underlying | risk-free rate).
        regex = r"(?P<T>[0-9]+(-[0-9]{2}){2} ([0-9]{2}:){2}[0-9]{2})|(?P<Sr>Unnamed: (?P<idx>[0-9]+))"
        match = re.match(regex, col_name)
        if not match:
            return col_name
        if match["T"]:
            return "T"
        elif match["Sr"]:
            col_idx = int(match["idx"])
            # Third last depicts the price of the underlying...
            if col_idx == ncol - 3:
                return "S"
            # ...and the second last the risk free rate.
            elif col_idx == ncol - 2:
                return "r"


In [5]:
data = OptionsData("isx2010C.xls")
data.get_sheet_names()

['isx15012010C',
 'isx19022010C',
 'isx17122010C',
 'isx19112010C',
 'isx15102010C',
 'isx17092010C',
 'isx20082010C',
 'isx16072010C',
 'isx18062010C',
 'isx21052010C',
 'isx16042010C',
 'isx19032010C']

A class encapsulating the Black-Scholes-Merton model and related computations, such as Greeks. Can create instances from `pd.Series` objects (as returned by pd.DataFrame.iterrows) via the `BSM.make_from_series` factory method.

In [6]:
@dataclass(frozen=True)
class BSM:
    S: float
    E: float
    r: float
    T: float
    C_obs: float
    sigma: float = 1.0
    d1: float = field(init=False)
    d2: float = field(init=False)
    
    def __post_init__(self):
        S, E, r, T, C_obs, sigma = self.S, self.E, self.r, self.T, self.C_obs, self.sigma
        sigma = BSMAux(S0=S, K=E, T=T, r=r, sigma=0.5, kind='call').implied_vol(C_obs)
        object.__setattr__(self, "sigma", sigma)
        eps = np.finfo(float).eps
        d1 = (math.log(S / E) + (r + 0.5 * self.sigma**2) * T) / (sigma * math.sqrt(T) + eps)
        object.__setattr__(self, "d1", d1)
        d2 = self.d1 - self.sigma * math.sqrt(T)
        object.__setattr__(self, "d2", d2)
        
    @staticmethod    
    def make_from_dict(d, E):
        return BSM(d["S"], int(E), d["r"], d["T_norm"], d[E])
    
    @staticmethod
    def make_from_series(ser, E, sigma=1.0):
        ser = ser.filter(regex=Util.make_regex_group_disjunction(["S", int(E), "r", "T_norm"]), axis="index")
        assert ser.shape[0] == 4, f"[{type(self).__name__}] Error: The Series should have an index of form [S, E, r, T_norm], got {ser.index}."
        S, r, T, C_obs = ser.array
        return BSM(S, E, r, T, C_obs, sigma=sigma)
    
    @functools.cached_property
    def delta(self):
        return norm.cdf(self.d1)
    
    @functools.cached_property
    def gamma(self):
        return norm.pdf(self.d1) / (self.S * self.sigma * math.sqrt(self.T))
    
    @functools.cached_property
    def theta(self):
        S, E, r, T, sigma, d1, d2 = self.S, self.E, self.r, self.T, self.sigma, self.d1, self.d2
        return -0.5 * S * norm.pdf(d1) * sigma / math.sqrt(T) - r * E * math.exp(-r * T) * norm.cdf(d2)
    
    @functools.cached_property
    def vega(self):
        return self.S * math.sqrt(self.T) * norm.pdf(self.d1)


In [7]:
test = data.get_df()
td = test.to_dict("index")
for t, row in td.items():
    print(f"Day {t}")
    strikes = list(filter(lambda x: re.match(r"[0-9]+", x), row.keys()))
    BSMs = {}
    for E in strikes:
        BSMs[E] = BSM.make_from_dict(row, E)
    greeks = pd.DataFrame(index=strikes)
    greeks["delta"] = pd.Series([BSMs[E].delta for E in greeks.index], index=greeks.index)
    greeks["gamma"] = pd.Series([BSMs[E].gamma for E in greeks.index], index=greeks.index)
    greeks["vega"] = pd.Series([BSMs[E].vega for E in greeks.index], index=greeks.index)
    print(greeks[greeks > 0.001])
    print("-" * 50)
    


[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'
Day 0
        delta     gamma        vega
340  0.979830       NaN   13.997796
345       NaN       NaN         NaN
350       NaN       NaN         NaN
355       NaN       NaN         NaN
360  0.968614       NaN   20.274473
365       NaN       NaN         NaN
370       NaN       NaN         NaN
375       NaN       NaN         NaN
380  0.951548  0.001256   28.869144
385       NaN       NaN         NaN
390       NaN       NaN         NaN
395       NaN       NaN         NaN
400  0.918179  0.001963   43.403146
405       NaN       NaN         NaN
410       NaN       NaN         NaN
415       NaN       NaN         NaN
420  0.872182  0.002876   60.011561
425       NaN       NaN         NaN
430       NaN       NaN         NaN
435       NaN       NaN         NaN
440  0.807413  0.004019   78.539065
445       NaN       NaN         NaN
450       NaN       NaN         NaN
455       NaN       NaN         NaN
460  0.716861 

/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in double_scalars
  + (self.r + 0.5 * self.sigma ** 2) * self.T
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (


        delta     gamma        vega
340       NaN       NaN         NaN
345       NaN       NaN         NaN
350       NaN       NaN         NaN
355       NaN       NaN         NaN
360  0.993640       NaN    5.003721
365       NaN       NaN         NaN
370       NaN       NaN         NaN
375       NaN       NaN         NaN
380  0.967253  0.001071   20.445721
385       NaN       NaN         NaN
390       NaN       NaN         NaN
395       NaN       NaN         NaN
400  0.923817  0.002027   40.037297
405       NaN       NaN         NaN
410       NaN       NaN         NaN
415       NaN       NaN         NaN
420  0.869055  0.003094   59.420608
425       NaN       NaN         NaN
430       NaN       NaN         NaN
435       NaN       NaN         NaN
440  0.794649  0.004331   79.489743
445       NaN       NaN         NaN
450       NaN       NaN         NaN
455       NaN       NaN         NaN
460  0.698419  0.005701   97.405723
465       NaN       NaN         NaN
470       NaN       NaN     

/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (


        delta     gamma        vega
340  0.967964       NaN   19.839047
345       NaN       NaN         NaN
350       NaN       NaN         NaN
355       NaN       NaN         NaN
360  0.956121  0.001045   25.649131
365       NaN       NaN         NaN
370       NaN       NaN         NaN
375       NaN       NaN         NaN
380  0.934839  0.001545   35.082123
385       NaN       NaN         NaN
390       NaN       NaN         NaN
395       NaN       NaN         NaN
400  0.900806  0.002275   48.178886
405       NaN       NaN         NaN
410       NaN       NaN         NaN
415       NaN       NaN         NaN
420  0.848366  0.003253   64.853662
425       NaN       NaN         NaN
430       NaN       NaN         NaN
435       NaN       NaN         NaN
440  0.775522  0.004435   82.712857
445       NaN       NaN         NaN
450       NaN       NaN         NaN
455       NaN       NaN         NaN
460  0.676900  0.005676   99.152175
465       NaN       NaN         NaN
470       NaN       NaN     

...

        delta     gamma       vega
340       NaN       NaN        NaN
345       NaN       NaN        NaN
350       NaN       NaN        NaN
355  0.985301       NaN   3.490108
360       NaN       NaN        NaN
365  0.984384       NaN   3.676371
370  0.983890       NaN   3.775834
375  0.797349       NaN  26.462956
380       NaN       NaN        NaN
385  0.983458       NaN   3.862459
390  0.981627       NaN   4.224668
395  0.980975       NaN   4.351980
400       NaN       NaN        NaN
405  0.980933       NaN   4.360161
410  0.980192       NaN   4.503632
415  0.979397       NaN   4.656375
420       NaN       NaN        NaN
425  0.977618       NaN   4.994063
430  0.976617  0.001005   5.181675
435  0.975527  0.001098   5.383979
440       NaN       NaN        NaN
445  0.973024  0.001327   5.841173
450       NaN       NaN        NaN
455  0.969963  0.001635   6.387492
460       NaN       NaN        NaN
465  0.963745  0.002147   7.458701
470  0.953319  0.002692   9.155070
475       NaN       

/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in doub

        delta     gamma       vega
340  0.985462       NaN   2.452007
345  0.985043       NaN   2.512609
350  0.984606       NaN   2.575592
355  0.984149       NaN   2.641127
360  0.983669       NaN   2.709402
365  0.983167       NaN   2.780625
370  0.982639       NaN   2.855026
375  0.982083       NaN   2.932861
380  0.981497       NaN   3.014414
385  0.980878       NaN   3.100003
390  0.980223       NaN   3.189984
395  0.979528       NaN   3.284758
400  0.978790       NaN   3.384779
405  0.978003       NaN   3.490560
410  0.977163       NaN   3.602691
415  0.976263       NaN   3.721843
420  0.975295       NaN   3.848795
425  0.974253       NaN   3.984447
430  0.973125  0.001074   4.129853
435  0.971900  0.001171   4.286251
440  0.970563  0.001281   4.455111
445  0.969097  0.001407   4.638188
450  0.967482  0.001553   4.837607
455  0.965691  0.001723   5.055959
460  0.957738  0.002087   5.993070
465  0.961441  0.002160   5.563121
470  0.958885  0.002447   5.861013
475  0.955953  0.002

/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in double_scalars
  + (self.r + 0.5 * self.sigma ** 2) * self.T
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: overflow encountered in double_scalars
  vol = vol + diff / opt.vega()


        delta     gamma       vega
340  0.987548       NaN   1.863559
345  0.987189       NaN   1.909896
350  0.986815       NaN   1.958064
355  0.986423       NaN   2.008194
360  0.986013       NaN   2.060431
365  0.985583       NaN   2.114935
370  0.985131       NaN   2.171883
375  0.984655       NaN   2.231472
380  0.984153       NaN   2.293920
385  0.983623       NaN   2.359470
390  0.983063       NaN   2.428397
395  0.982469       NaN   2.501009
400  0.981837       NaN   2.577654
405  0.981165       NaN   2.658726
410  0.980446       NaN   2.744676
415  0.979677       NaN   2.836021
420  0.978851       NaN   2.933356
425  0.977960       NaN   3.037370
430  0.976998       NaN   3.148870
435  0.975953  0.001045   3.268803
440  0.974813  0.001143   3.398290
445  0.973565  0.001256   3.538671
450  0.972190  0.001385   3.691564
455  0.970667  0.001536   3.858941
460  0.968969  0.001714   4.043233
465  0.967061  0.001924   4.247483
470  0.964898  0.002177   4.475553
475  0.962421  0.002

In [8]:
t = 0
strike_step = 5
S = td[t]["S"]
S
strikes = list(filter(lambda x: re.match(r"\d+", x), td[t].keys()))
strike_to_buy = strikes[np.argmin(np.abs(np.array(strikes, dtype=np.int64) - S))]
strike_to_buy
Gs = greeks[greeks > 0.001].dropna(how='all')
strike_to_buy
Gs = Gs.fillna(0)
print(Gs)
# G*w = p
p = Gs.loc[strike_to_buy].to_numpy()[0]
G = p
res = solve(G, p)
p - res*p

        delta     gamma        vega
340  0.979830  0.000000   13.997796
360  0.968614  0.000000   20.274473
380  0.951548  0.001256   28.869144
400  0.918179  0.001963   43.403146
420  0.872182  0.002876   60.011561
440  0.807413  0.004019   78.539065
460  0.716861  0.005282   97.142809
480  0.600874  0.006448  110.829038
500  0.463360  0.007058  114.026100
520  0.320293  0.006735  102.685257


KeyError: '490'

In [140]:

eps = np.finfo(float).eps

class Hedger:
    @dataclass
    class HedgingStats:
        cost_basis: float
        mse: float = 0.0
        total_cost: float = 0.0
            
        def __repr__(self):
            return (f"[Hedger.{type(self).__name__}]: "
                    f"Assuming a cost basis of {self.cost_basis*100:.2f}%, mean-squared error "
                    f"of hedging was {self.mse:.2f}, and the total costs were ${self.total_cost:.2f}.")
    
    @dataclass
    class DeltaState:
        long: float
        short: float
        delta: float
            
    @dataclass
    class DeltaVegaState:
        portfolio: float
        underlying: float
        rep_option: float
        alpha: float
        eta: float

    def delta_hedge(self, data, sheet_name="", portfolio_size=2, schedule=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, strikes_considered.index))}"))
        
        # Compute and save state for the required computations.
        BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in strikes_considered.index}
        deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
        longs = day0[strikes_considered.index].sum()
        state_prev = Hedger.DeltaState(longs, deltas * day0.S, deltas)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.short
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            BSMs = {E: BSM.make_from_dict(row, E) for E in strikes_considered.index}
            longs = sum(map(lambda x: x.C_obs, BSMs.values()))
            state = Hedger.DeltaState(longs, state_prev.delta * row["S"], state_prev.delta)
            dlong = state.long - state_prev.long
            dshort = state.short - state_prev.short
            squared_errors.append((dlong - dshort)**2)
            # Rehedge?
            if t % schedule == 0:
                deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
                state.short = deltas * row["S"]
                state.delta = deltas
                stats.total_cost += abs(cost_basis * (state_prev.delta - state.delta) * row["S"])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        
    def delta_vega_hedge(self, data, schedule=2, sheet_name="", sheet_name2="isx19022010C", portfolio_size=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        df2 = data.get_df(sheet_name=sheet_name2)
        n_options_considered = 2
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size+1]]
        rep_option_strike = strikes_considered[-1:]
        portfolio_strikes = strikes_considered[:-1]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, portfolio_strikes.index))}"))
        
        # Compute and save state for the required computations.
        portfolio_BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in portfolio_strikes.index}
        rep_E = rep_option_strike.index[0]
        rep_option_BSM = BSM(day0.S, int(rep_E), day0.r, day0.T_norm, day0[rep_E])
        portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
        portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
        rep_option_delta = rep_option_BSM.delta
        rep_option_vega = rep_option_BSM.vega
        portfolio = day0[portfolio_strikes.index].sum() 
        alpha = -portfolio_delta + portfolio_vega / rep_option_vega * rep_option_delta
        eta = -portfolio_vega / rep_option_vega
        # don't hedge at day 0 if some delta or vega is nan (=implied vol is nan)
        if math.isnan(alpha+eta):
            alpha, eta = 0, 0
        state_prev = Hedger.DeltaVegaState(portfolio, alpha * day0.S, eta * day0[rep_E], alpha, eta)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.alpha * day0.S + cost_basis * state_prev.eta * day0[rep_E]
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            portfolio_BSMs = {E: BSM.make_from_dict(row, E) for E in portfolio_strikes.index}
            rep_option_BSM = BSM.make_from_dict(row, rep_E)
            portfolio = sum(map(lambda x: x.C_obs, portfolio_BSMs.values()))
            state = Hedger.DeltaVegaState(portfolio, state_prev.alpha * row["S"], state_prev.eta * row[rep_E], state_prev.alpha, state_prev.eta)
            diff = (state.portfolio + state.underlying + state.rep_option) - (state_prev.portfolio + state_prev.underlying + state_prev.rep_option)
            squared_errors.append((diff)**2)
            # Rehedge?
            if t % schedule == 0:
                portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
                portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
                rep_option_delta = rep_option_BSM.delta
                rep_option_vega = rep_option_BSM.vega
                alpha = -portfolio_delta + portfolio_vega / (rep_option_vega + eps) * rep_option_delta
                eta = -portfolio_vega / (rep_option_vega + eps)
                if not math.isnan(alpha+eta):
                    state.alpha = alpha
                    state.eta = eta
                state.underlying = state.alpha * row["S"]
                state.rep_option = state.eta * row[rep_E]
                stats.total_cost += abs(cost_basis * (state_prev.alpha - state.alpha) * row["S"] + cost_basis * (state_prev.eta - state.eta) * row[rep_E])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats

In [149]:
# Doing replicated option from another sheet here

'''

eps = np.finfo(float).eps

class Hedger:
    @dataclass
    class HedgingStats:
        cost_basis: float
        mse: float = 0.0
        total_cost: float = 0.0
            
        def __repr__(self):
            return (f"[Hedger.{type(self).__name__}]: "
                    f"Assuming a cost basis of {self.cost_basis*100:.2f}%, mean-squared error "
                    f"of hedging was {self.mse:.2f}, and the total costs were ${self.total_cost:.2f}.")
    
    @dataclass
    class DeltaState:
        long: float
        short: float
        delta: float
            
    @dataclass
    class DeltaVegaState:
        portfolio: float
        underlying: float
        rep_option: float
        alpha: float
        eta: float

    def delta_hedge(self, data, sheet_name="", portfolio_size=2, schedule=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, strikes_considered.index))}"))
        
        # Compute and save state for the required computations.
        BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in strikes_considered.index}
        deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
        longs = day0[strikes_considered.index].sum()
        state_prev = Hedger.DeltaState(longs, deltas * day0.S, deltas)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.short
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            BSMs = {E: BSM.make_from_dict(row, E) for E in strikes_considered.index}
            longs = sum(map(lambda x: x.C_obs, BSMs.values()))
            state = Hedger.DeltaState(longs, state_prev.delta * row["S"], state_prev.delta)
            dlong = state.long - state_prev.long
            dshort = state.short - state_prev.short
            squared_errors.append((dlong - dshort)**2)
            # Rehedge?
            if t % schedule == 0:
                deltas = np.sum(np.nan_to_num(list(map(lambda x: x.delta, BSMs.values()))))
                state.short = deltas * row["S"]
                state.delta = deltas
                stats.total_cost += abs(cost_basis * (state_prev.delta - state.delta) * row["S"])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        
    def delta_vega_hedge(self, data, schedule=2, sheet_name="", sheet_name2="isx19022010C", portfolio_size=2, cost_basis=0.01):
        df = data.get_df(sheet_name=sheet_name)
        df2 = data.get_df(sheet_name=sheet_name2)
        
        # We consider at-the-money options.
        day0 = df.iloc[0]
        strikes = day0.dropna().filter(regex=r"\d+").index
        option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
        day0_2 = df2.iloc[0]
        strikes2 = day0_2.dropna().filter(regex=r"\d+").index
        option_value_ser_2 = pd.Series(data=strikes2, index=strikes2, dtype=int).apply(lambda E: abs(day0_2.S - E))
        
        portfolio_strikes = option_value_ser.iloc[np.argsort(option_value_ser)[:portfolio_size]]
        rep_option_strike = option_value_ser_2.iloc[np.argsort(option_value_ser_2)[:1]]
        print((f"[{type(self).__name__}] Info: "
               "Considering a position in call(s) with strike price(s) of "
               f"{', '.join(map(lambda x: '$' + x, portfolio_strikes.index))}"))
        
        # Compute and save state for the required computations.
        portfolio_BSMs = {E: BSM(day0.S, int(E), day0.r, day0.T_norm, day0[E]) for E in portfolio_strikes.index}
        rep_E = rep_option_strike.index[0]
        rep_option_BSM = BSM(day0_2.S, int(rep_E), day0_2.r, day0_2.T_norm, day0_2[rep_E])
        portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
        portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
        rep_option_delta = rep_option_BSM.delta
        rep_option_vega = rep_option_BSM.vega
        portfolio = day0[portfolio_strikes.index].sum() 
        alpha = -portfolio_delta + portfolio_vega / rep_option_vega * rep_option_delta
        eta = -portfolio_vega / rep_option_vega
        # don't hedge at day 0 if some delta or vega is nan (=implied vol is nan)
        if math.isnan(alpha+eta):
            alpha, eta = 0, 0
        state_prev = Hedger.DeltaVegaState(portfolio, alpha * day0.S, eta * day0_2[rep_E], alpha, eta)
        stats = Hedger.HedgingStats(cost_basis=cost_basis)
        stats.total_cost += cost_basis * state_prev.alpha * day0.S + cost_basis * state_prev.eta * day0_2[rep_E]
        
        # Simulate trading with the provided data and perform hedging.
        day1_onwards = df.iloc[1:-1].to_dict("index")
        day1_onwards_2 = df2.iloc[1:-1].to_dict("index")
        squared_errors = []
        for t, row in day1_onwards.items():
            row2 = day1_onwards_2[t]
            portfolio_BSMs = {E: BSM.make_from_dict(row, E) for E in portfolio_strikes.index}
            rep_option_BSM = BSM.make_from_dict(row2, rep_E)
            portfolio = sum(map(lambda x: x.C_obs, portfolio_BSMs.values()))
            state = Hedger.DeltaVegaState(portfolio, state_prev.alpha * row["S"], state_prev.eta * row2[rep_E], state_prev.alpha, state_prev.eta)
            diff = (state.portfolio + state.underlying + state.rep_option) - (state_prev.portfolio + state_prev.underlying + state_prev.rep_option)
            squared_errors.append((diff)**2)
            # Rehedge?
            if t % schedule == 0:
                portfolio_delta = np.sum(list(map(lambda x: x.delta, portfolio_BSMs.values())))
                portfolio_vega = np.sum(list(map(lambda x: x.vega, portfolio_BSMs.values())))
                rep_option_delta = rep_option_BSM.delta
                rep_option_vega = rep_option_BSM.vega
                alpha = -portfolio_delta + portfolio_vega / (rep_option_vega + eps) * rep_option_delta
                eta = -portfolio_vega / (rep_option_vega + eps)
                if not math.isnan(alpha+eta):
                    state.alpha = alpha
                    state.eta = eta
                state.underlying = state.alpha * row["S"]
                state.rep_option = state.eta * row2[rep_E]
                stats.total_cost += abs(cost_basis * (state_prev.alpha - state.alpha) * row["S"] + cost_basis * (state_prev.eta - state.eta) * row2[rep_E])
            state_prev = state
                
        stats.mse = np.mean(squared_errors)
        return stats
        '''

In [150]:
not math.isnan(nan)

False

In [151]:
a, x = 0, 0

In [152]:
h = Hedger()
h.delta_vega_hedge(data, sheet_name='isx15012010C', sheet_name2='isx19022010C', portfolio_size=2)
#h.delta_hedge(data, portfolio_size=2)

[Hedger] Info: Considering a position in call(s) with strike price(s) of $500, $480


[Hedger.HedgingStats]: Assuming a cost basis of 1.00%, mean-squared error of hedging was 41.02, and the total costs were $39.83.

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'
[Hedger] Info: Considering a position in call(s) with strike price(s) of $500


IndexError: invalid index to scalar variable.

In [39]:
df = data.get_df(sheet_name="")
        
# We consider at-the-money options.
day0 = df.iloc[0]
strikes = day0.dropna().filter(regex=r"\d+").index
option_value_ser = pd.Series(data=strikes, index=strikes, dtype=int).apply(lambda E: abs(day0.S - E))
strikes_considered = option_value_ser.iloc[np.argsort(option_value_ser)[:5]]
rep_option_strike = strikes_considered[-1:]
portfolio_strikes = strikes_considered[:-1]

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'


In [58]:
day1_onwards = df.iloc[1:-1].to_dict("index")
for t, row in day1_onwards.items():
    break

In [146]:
day1_onwards[1]

{'T': 85,
 'T_norm': 0.3373015873015873,
 'S': 494.35,
 'r': 0.0011,
 '340': 155.05,
 '345': nan,
 '350': nan,
 '355': nan,
 '360': 135.35,
 '365': nan,
 '370': nan,
 '375': nan,
 '380': 115.9,
 '385': nan,
 '390': nan,
 '395': nan,
 '400': 96.9,
 '405': nan,
 '410': nan,
 '415': nan,
 '420': 78.75,
 '425': nan,
 '430': nan,
 '435': nan,
 '440': 61.45,
 '445': nan,
 '450': nan,
 '455': nan,
 '460': 45.45,
 '465': nan,
 '470': nan,
 '475': nan,
 '480': 31.4,
 '485': nan,
 '490': nan,
 '495': nan,
 '500': 19.8,
 '505': nan,
 '510': nan,
 '515': nan,
 '520': 11.15,
 '525': nan,
 '530': nan,
 '535': nan,
 '540': nan,
 '545': nan,
 '550': nan,
 '555': nan,
 '560': nan,
 '565': nan,
 '570': nan}

In [61]:
rep_E = rep_option_strike.index[0]
row[rep_E]

61.45

In [68]:
day0[rep_E]


59.25

In [55]:
day0[rep_option_strike.index][0]

59.25

In [49]:
8.66+11.34+28.66+31.34

80.0

In [50]:
portfolio_strikes

500     8.66
480    11.34
520    28.66
460    31.34
dtype: float64

In [63]:
E = 500
schedule = 2
df = data.get_df(E).dropna()

import time

t0 = time.time()
# t = 0:
rows_iterator = df.iterrows()
_, row = next(rows_iterator)
bsm_prev = BSM.make_from_series(row, E)
long_prev = bsm_prev.C_obs
delta_factor = bsm_prev.delta
short_prev = delta_factor * bsm_prev.S

# 0 < t < T:
mse = 0.0
for t, row in rows_iterator:
    bsm = BSM.make_from_series(row, E)
    long = bsm.C_obs
    dlong = long - long_prev
    short = delta_factor * bsm.S
    dshort = short - short_prev
    mse += (dlong - dshort)**2
    long_prev = long
    bsm_prev = bsm
    # Rehedge?
    if t % schedule == 0:
        delta_factor = bsm.delta
        short_prev = delta_factor * bsm.S
    else:
        short_prev = short

mse /= df.shape[0]

t1 = time.time()
print(f"Took {(t1 - t0)*1000:.2f} ms")

[OptionsData] Warning: sheet name not specified; proceeding with 'isx15012010C'


/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (


Took 4059.20 ms


In [64]:
print(f"Single option delta hedging {mse=:.2f}")

Single option delta hedging mse=nan
